# Load Dataset

In [1]:
!pip install datasets --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [2]:
from datasets import load_dataset

In [3]:
Dataset = load_dataset("Bahareh0281/liveness_images")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6427 [00:00<?, ? examples/s]

In [6]:
Dataset['train'][7]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256>,
 'label': 0}

# Import Necessary Libraries

In [4]:
import cv2
import numpy as np
import os
from skimage.feature import local_binary_pattern
from skimage import measure
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Feature Extraction Functions

In [5]:
radius = 3
n_points = 8 * radius

def compute_fourier_transform(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift))
    return magnitude_spectrum

def compute_lbp(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, n_points, radius, method="uniform")
    return lbp

def compute_depth(image):
    depth = image[:, :, 2]
    return depth

def extract_statistical_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mean = np.mean(gray)
    std_dev = np.std(gray)
    skewness = np.mean((gray - mean) ** 3) / (std_dev ** 3)
    kurtosis = np.mean((gray - mean) ** 4) / (std_dev ** 4)
    entropy = measure.shannon_entropy(gray)
    return mean, std_dev, skewness, kurtosis, entropy

# Preprocess Input Images

In [6]:
len(Dataset['train'])

6427

In [8]:
def process_images(dataset, traget_size, num=0):
    train_images_features = []
    train_images_labels = []
    if num == 0:
        num = len(dataset['train'])

    for i in range(num):
        img = dataset['train'][i]['image']
        if isinstance(img, Image.Image):
            img = np.array(img)  # Convert PIL image to NumPy array

            # Extract frequency features
            magnitude_spectrum = compute_fourier_transform(img)
            magnitude_spectrum_resized = cv2.resize(magnitude_spectrum, (traget_size, traget_size))

            # Extract LBP features
            lbp = compute_lbp(img)
            lbp_hist, _ = np.histogram(lbp, bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
            lbp_hist_normalized = lbp_hist / lbp_hist.sum()
            lbp_hist_resized = cv2.resize(lbp_hist_normalized.reshape(-1, 1), (traget_size, traget_size))

            # Extract statistical features
            mean, std_dev, skewness, kurtosis, entropy = extract_statistical_features(img)
            statistical_features = np.array([mean, std_dev, skewness, kurtosis, entropy])
            statistical_features_resized = cv2.resize(statistical_features.reshape(-1, 1), (traget_size, traget_size))

            # Combine features into a 3D array
            combined_features = np.stack([
                magnitude_spectrum_resized,
                lbp_hist_resized,
                statistical_features_resized
            ], axis=-1)

            train_images_features.append(combined_features)
            train_images_labels.append(dataset['train'][i]['label'])

    return np.array(train_images_features), np.array(train_images_labels)

In [9]:
train_images_features, train_images_labels = process_images(Dataset, 64, 3000)

<ipython-input-5-719dd35ae871>:8: RuntimeWarning: divide by zero encountered in log
  magnitude_spectrum = 20 * np.log(np.abs(fshift))


In [15]:
len(train_images_features)

3000

In [16]:
train_images_features.shape

(3000, 64, 64, 3)

In [17]:
train_images_features[1]

array([[[9.98781304e+01, 1.70135498e-02, 1.23999863e+02],
        [1.03112881e+02, 1.70135498e-02, 1.23999863e+02],
        [1.00401371e+02, 1.70135498e-02, 1.23999863e+02],
        ...,
        [8.08424041e+01, 1.70135498e-02, 1.23999863e+02],
        [8.49438029e+01, 1.70135498e-02, 1.23999863e+02],
        [9.55299601e+01, 1.70135498e-02, 1.23999863e+02]],

       [[7.85450649e+01, 1.65896416e-02, 1.23999863e+02],
        [8.88281131e+01, 1.65896416e-02, 1.23999863e+02],
        [8.47440767e+01, 1.65896416e-02, 1.23999863e+02],
        ...,
        [9.16315104e+01, 1.65896416e-02, 1.23999863e+02],
        [9.06426121e+01, 1.65896416e-02, 1.23999863e+02],
        [8.25498088e+01, 1.65896416e-02, 1.23999863e+02]],

       [[9.15188864e+01, 1.50151253e-02, 1.23999863e+02],
        [9.45883978e+01, 1.50151253e-02, 1.23999863e+02],
        [9.23813503e+01, 1.50151253e-02, 1.23999863e+02],
        ...,
        [9.37271837e+01, 1.50151253e-02, 1.23999863e+02],
        [9.23618173e+01, 1.50

In [18]:
len(train_images_features[1])

64

In [19]:
len(train_images_features[0])

64

# Split training dataset and prepare it for train process

In [20]:
# Convert depth features to a numpy array
features = np.array(train_images_features)
labels = np.array(train_images_labels)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)


# Create CNN Model and Train it

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20
60/60 [==============================] - 6s 12ms/step - loss: 6.0613 - accuracy: 0.7302 - val_loss: 0.4718 - val_accuracy: 0.8271
Epoch 2/20
60/60 [==============================] - 0s 6ms/step - loss: 0.4913 - accuracy: 0.7891 - val_loss: 0.4242 - val_accuracy: 0.8271
Epoch 3/20
60/60 [==============================] - 0s 6ms/step - loss: 0.4717 - accuracy: 0.7948 - val_loss: 0.4210 - val_accuracy: 0.8271
Epoch 4/20
60/60 [==============================] - 0s 6ms/step - loss: 0.4624 - accuracy: 0.7948 - val_loss: 0.4058 - val_accuracy: 0.8271
Epoch 5/20
60/60 [==============================] - 0s 6ms/step - loss: 0.4619 - accuracy: 0.7948 - val_loss: 0.4135 - val_accuracy: 0.8271
Epoch 6/20
60/60 [==============================] - 0s 7ms/step - loss: 0.4569 - accuracy: 0.7969 - val_loss: 0.3923 - val_accuracy: 0.8271
Epoch 7/20
60/60 [==============================] - 0s 7ms/step - loss: 0.4495 - accuracy: 0.8036 - val_loss: 0.3847 - val_accuracy: 0.8417
Epoch 8/20
60/60 [=

# Evaluate on dataset tests

In [22]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


19/19 [==============================] - 0s 11ms/step - loss: 0.6804 - accuracy: 0.8100
Test Accuracy: 81.00%


# Load test videos

In [23]:
import gdown

file_id = '1a5R5h05hCyw9PzIBhSjy2jLL3dSFy2xA'
destination = '/content/dataset.zip'  # Path where the file will be saved
gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=False)

import zipfile

with zipfile.ZipFile(destination, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

Downloading...
From (original): https://drive.google.com/uc?id=1a5R5h05hCyw9PzIBhSjy2jLL3dSFy2xA
From (redirected): https://drive.google.com/uc?id=1a5R5h05hCyw9PzIBhSjy2jLL3dSFy2xA&confirm=t&uuid=a5914a2e-aa94-4d78-a707-c78c9ec58d09
To: /content/dataset.zip
100%|██████████| 377M/377M [00:03<00:00, 101MB/s]


# Generate random frames from each video

In [24]:
import random

def extract_frames(video_path, save_path, label, test):
    # Open the video file
    video = cv2.VideoCapture(video_path)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Select one random frame
    random_frame = random.randint(0, frame_count - 1)

    # Set the position of the video to the selected frame
    video.set(cv2.CAP_PROP_POS_FRAMES, random_frame)
    success, frame = video.read()

    # If the frame was successfully read, save it
    if success:
        frame_path = os.path.join(save_path, f"{label}_{random_frame}.jpg")
        cv2.imwrite(frame_path, frame)

        # Convert the frame to a PIL image
        pil_image = Image.open(frame_path)

        # Save the image and label to the dictionary
        test.append({'image': pil_image, 'label': label})

    # Release the video file
    video.release()

In [25]:
import cv2

fake_test_videos_path = '/content/dataset/fake/test'
real_test_videos_path = '/content/dataset/real/test'

save_frames_path = '/content/extracted_frames/test'
# Create the directory if it doesn't exist
if not os.path.exists(save_frames_path):
    os.makedirs(save_frames_path)

# Create a list to hold the dictionary entries
test = []

# Iterate over fake videos and extract frames
for fake_video_file in os.listdir(fake_test_videos_path):
    fake_video_path = os.path.join(fake_test_videos_path, fake_video_file)
    extract_frames(fake_video_path, save_frames_path, 0, test)

# Iterate over real videos and extract frames
for real_video_file in os.listdir(real_test_videos_path):
    real_video_path = os.path.join(real_test_videos_path, real_video_file)
    extract_frames(real_video_path, save_frames_path, 1, test)

In [26]:
len(test)

33

# Extract features from each frame

In [27]:
def process_images_2(tests, traget_size):
    test_images_features = []
    test_images_labels = []


    for i in range(len(tests)):
        img = tests[i]['image']
        if isinstance(img, Image.Image):
            img = np.array(img)  # Convert PIL image to NumPy array

            # Extract frequency features
            magnitude_spectrum = compute_fourier_transform(img)
            magnitude_spectrum_resized = cv2.resize(magnitude_spectrum, (traget_size, traget_size))

            # Extract LBP features
            lbp = compute_lbp(img)
            lbp_hist, _ = np.histogram(lbp, bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
            lbp_hist_normalized = lbp_hist / lbp_hist.sum()
            lbp_hist_resized = cv2.resize(lbp_hist_normalized.reshape(-1, 1), (traget_size, traget_size))

            # Extract statistical features
            mean, std_dev, skewness, kurtosis, entropy = extract_statistical_features(img)
            statistical_features = np.array([mean, std_dev, skewness, kurtosis, entropy])
            statistical_features_resized = cv2.resize(statistical_features.reshape(-1, 1), (traget_size, traget_size))

            # Combine features into a 3D array
            combined_features = np.stack([
                magnitude_spectrum_resized,
                lbp_hist_resized,
                statistical_features_resized
            ], axis=-1)

            test_images_features.append(combined_features)
            test_images_labels.append(tests[i]['label'])

    return np.array(test_images_features), np.array(test_images_labels)


# Extract features from each frame and convert labels to one-hot form

In [29]:
test_frames_features, test_frames_labels = process_images_2(test, 64)
test_frames_labels = to_categorical(test_frames_labels, num_classes=2)

In [30]:
test_frames_features.shape

(33, 64, 64, 3)

In [31]:
test_frames_labels.shape

(33, 2)

# Evaluate model on test set

In [32]:
test_loss, test_accuracy = model.evaluate(test_frames_features, test_frames_labels)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

2/2 [==============================] - 0s 168ms/step - loss: 1.5870 - accuracy: 0.6364
Test Accuracy: 63.64%


In [39]:
# Make predictions
paths = []
print(save_frames_path)
for root, _, files in os.walk(save_frames_path):
    for file in files:
        paths.append(os.path.join(root, file))

print(len(paths))
y_pred_proba = model.predict(test_frames_features)
predictions_for_frames = []
# Output the prediction vector for each test image
for idx, prediction_vector in enumerate(y_pred_proba):
    print(f"{paths[idx]}: {prediction_vector}")
    predictions_for_frames.append((paths[idx], prediction_vector))


/content/extracted_frames/test
31
2/2 [==============================] - 0s 4ms/step
/content/extracted_frames/test/0_30.jpg: [0.6499538 0.3500462]
/content/extracted_frames/test/1_25.jpg: [0.96498835 0.03501165]
/content/extracted_frames/test/0_71.jpg: [0.9688845  0.03111554]
/content/extracted_frames/test/1_83.jpg: [0.7049787  0.29502124]
/content/extracted_frames/test/0_32.jpg: [0.94680214 0.05319795]
/content/extracted_frames/test/1_3.jpg: [0.9315848  0.06841516]
/content/extracted_frames/test/1_23.jpg: [9.9995363e-01 4.6319627e-05]
/content/extracted_frames/test/0_70.jpg: [0.8605035  0.13949648]
/content/extracted_frames/test/1_0.jpg: [0.87676734 0.12323274]
/content/extracted_frames/test/0_42.jpg: [0.81033164 0.18966833]
/content/extracted_frames/test/0_34.jpg: [0.96459174 0.03540822]
/content/extracted_frames/test/0_45.jpg: [0.9151475  0.08485247]
/content/extracted_frames/test/0_31.jpg: [9.9936217e-01 6.3780544e-04]
/content/extracted_frames/test/0_12.jpg: [0.9466931  0.0533069

IndexError: list index out of range

# InceptionV3


In [96]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.preprocessing.image import load_img

In [120]:
train_images_features, train_images_labels = process_images(Dataset, 75, 3000)

Exception ignored in: <function _xla_gc_callback at 0x7c3811b96200>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [99]:
# Convert depth features to a numpy array
features2 = np.array(train_images_features)
labels2 = np.array(train_images_labels)
# Split the data into training and testing sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(features2, labels2, test_size=0.2, random_state=42)

# One-hot encode the labels
y_train2 = to_categorical(y_train2, num_classes=2)
y_test2 = to_categorical(y_test2, num_classes=2)

In [100]:
# Load the pre-trained model
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(75, 75, 3))  # Exclude the top layer

# Freeze the layers of the pre-trained model
for layer in inception_model.layers:
    layer.trainable = False

# Add a global spatial average pooling layer
x = inception_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer with the number of classes you have (binary classification)
predictions = Dense(2, activation='softmax')(x)

# This is the model we will train
model2 = Model(inputs=inception_model.input, outputs=predictions)

87910968/87910968 [==============================] - 1s 0us/step


In [101]:
# Compile the model
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model2.fit(X_train2, y_train2, epochs=10, batch_size=32, validation_data=(X_test2, y_test2))

Epoch 1/10
75/75 [==============================] - 12s 70ms/step - loss: 10.6319 - accuracy: 0.7254 - val_loss: 4.1390 - val_accuracy: 0.5717
Epoch 2/10
75/75 [==============================] - 1s 18ms/step - loss: 1.6047 - accuracy: 0.7804 - val_loss: 1.3298 - val_accuracy: 0.8350
Epoch 3/10
75/75 [==============================] - 1s 19ms/step - loss: 1.1379 - accuracy: 0.7971 - val_loss: 3.1350 - val_accuracy: 0.4800
Epoch 4/10
75/75 [==============================] - 1s 19ms/step - loss: 1.6602 - accuracy: 0.7892 - val_loss: 0.5015 - val_accuracy: 0.8700
Epoch 5/10
75/75 [==============================] - 2s 27ms/step - loss: 0.9463 - accuracy: 0.7958 - val_loss: 1.7081 - val_accuracy: 0.8133
Epoch 6/10
75/75 [==============================] - 1s 18ms/step - loss: 0.7672 - accuracy: 0.8167 - val_loss: 0.8685 - val_accuracy: 0.8533
Epoch 7/10
75/75 [==============================] - 1s 18ms/step - loss: 0.4900 - accuracy: 0.8417 - val_loss: 0.7989 - val_accuracy: 0.6567
Epoch 8/10


In [102]:
# unfreeze the layers of the pre-trained model
for layer in model2.layers:
    layer.trainable = True

In [103]:
# Compile the model
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model2.fit(X_train2, y_train2, epochs=10, batch_size=32, validation_data=(X_test2, y_test2))

Epoch 1/10
75/75 [==============================] - 40s 86ms/step - loss: 0.5452 - accuracy: 0.7925 - val_loss: 1.8175 - val_accuracy: 0.8100
Epoch 2/10
75/75 [==============================] - 4s 55ms/step - loss: 0.3496 - accuracy: 0.8650 - val_loss: 1.6700 - val_accuracy: 0.8100
Epoch 3/10
75/75 [==============================] - 5s 67ms/step - loss: 0.2547 - accuracy: 0.9038 - val_loss: 1.6330 - val_accuracy: 0.8100
Epoch 4/10
75/75 [==============================] - 4s 54ms/step - loss: 0.2465 - accuracy: 0.9108 - val_loss: 0.5524 - val_accuracy: 0.8467
Epoch 5/10
75/75 [==============================] - 4s 55ms/step - loss: 0.2207 - accuracy: 0.9287 - val_loss: 0.7412 - val_accuracy: 0.8117
Epoch 6/10
75/75 [==============================] - 5s 68ms/step - loss: 0.1640 - accuracy: 0.9504 - val_loss: 0.4944 - val_accuracy: 0.8167
Epoch 7/10
75/75 [==============================] - 4s 55ms/step - loss: 0.1528 - accuracy: 0.9425 - val_loss: 0.2576 - val_accuracy: 0.8817
Epoch 8/10
7

In [104]:
# Evaluate the model
test_loss, test_accuracy = model2.evaluate(X_test2, y_test2)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

19/19 [==============================] - 1s 25ms/step - loss: 0.1973 - accuracy: 0.9217
Test Loss: 0.19729502499103546, Test Accuracy: 0.92166668176651


In [105]:
test_frames_features2, test_frames_labels2 = process_images_2(test, 75)
test_frames_labels2 = to_categorical(test_frames_labels2, num_classes=2)

In [107]:
test_loss, test_accuracy = model.evaluate(test_frames_features2, test_frames_labels2)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

1/1 [==============================] - 0s 48ms/step - loss: 1.2868 - accuracy: 0.5938
Test Accuracy: 59.38%


In [109]:
# Make predictions

y_pred_proba = model2.predict(test_frames_features2)
predictions_for_frames = []
# Output the prediction vector for each test image
for idx, prediction_vector in enumerate(y_pred_proba):
    print(f"Prediction vector for test image {idx+1}: {prediction_vector}")
    predictions_for_frames.append((idx, prediction_vector))


1/1 [==============================] - 1s 1s/step
Prediction vector for test image 1: [0.9574484 0.0425516]
Prediction vector for test image 2: [0.98669654 0.01330348]
Prediction vector for test image 3: [0.9791821  0.02081785]
Prediction vector for test image 4: [0.19328159 0.8067184 ]
Prediction vector for test image 5: [0.97900987 0.02099015]
Prediction vector for test image 6: [0.987206   0.01279405]
Prediction vector for test image 7: [0.40649554 0.5935044 ]
Prediction vector for test image 8: [0.90500516 0.09499477]
Prediction vector for test image 9: [0.9377996  0.06220046]
Prediction vector for test image 10: [0.9688701  0.03112992]
Prediction vector for test image 11: [0.9831234  0.01687652]
Prediction vector for test image 12: [0.98220485 0.0177952 ]
Prediction vector for test image 13: [0.9722124  0.02778759]
Prediction vector for test image 14: [0.6606057 0.3393943]
Prediction vector for test image 15: [0.45547906 0.5445209 ]
Prediction vector for test image 16: [0.971383  

# Crop

In [110]:
!pip install mtcnn --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.1 MB/s eta 0:00:00


In [111]:
from mtcnn import MTCNN

In [112]:
test_images_dir = '/content/extracted_frames2/test'

In [113]:
def load_image_paths(directory):
    image_paths = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_paths.append(os.path.join(directory, filename))
    return image_paths

In [114]:
def detect_and_crop_faces(image_path, detector):
    image = Image.open(image_path)
    image_np = np.asarray(image)
    result = detector.detect_faces(image_np)
    if result:
        for person in result:
            bounding_box = person['box']
            keypoints = person['keypoints']

            # Crop the detected face
            x, y, width, height = bounding_box
            cropped_face = image_np[y:y+height, x:x+width]

            # Convert the cropped face back to an image
            cropped_face_image = Image.fromarray(cropped_face)

            return cropped_face_image
    return None

In [115]:
# Initialize the MTCNN face detector
detector = MTCNN()

In [116]:
# Directory containing the original images
test_image_paths = load_image_paths(test_images_dir)

# Directory to save cropped face images
output_dir = '/content/cropped_faces/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each image and save the cropped face with the original filename
for image_path in test_image_paths:
    cropped_face = detect_and_crop_faces(image_path, detector)
    # print("Image path is:", test_image_paths)
    if cropped_face:
        # Extract the original filename
        original_filename = os.path.basename(image_path)
        # Save the cropped face image with the original filename
        cropped_face.save(os.path.join(output_dir, original_filename))

1/1 [==============================] - 1s 521ms/step


1/1 [==============================] - 0s 17ms/step


In [117]:
# Function to resize images in a directory
def resize_images(directory, target_size=(64, 64)):
    resized_images = []
    test_image_paths = []
    test_image_labels = []

    # Iterate over each image in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            label = filename.split('.')[0].split('_')[0]
            image_path = os.path.join(directory, filename)
            image = cv2.imread(image_path)
            resized_image = cv2.resize(image, target_size)

            # Append resized image, image path, and label
            resized_images.append(resized_image)
            test_image_paths.append(image_path)
            test_image_labels.append(int(label))  # Convert label to int

    # Convert lists to numpy arrays
    resized_images = np.array(resized_images)
    test_image_labels = np.array(test_image_labels)

    return resized_images, test_image_paths, test_image_labels

In [118]:
# Resize images in test directory
cropped_resized_images, cropped_test_image_paths, cropped_test_image_labels = resize_images(output_dir)

# Check the shape of resized images
print("Resized images shape:", cropped_resized_images.shape)

Resized images shape: (32, 64, 64, 3)


In [119]:
# Convert labels to one-hot encoding
cropped_test_image_labels_onehot = to_categorical(cropped_test_image_labels, num_classes=2)

In [ ]:
train_images_features, train_images_labels = process_images_2(cropped_test_images, 64)

In [ ]:
# Evaluate the model
cropped_test_loss, cropped_test_accuracy = model.evaluate(cropped_resized_images, cropped_test_image_labels_onehot)

In [ ]:
# Predictions
cropped_predictions = model.predict(cropped_resized_images)
cropped_predicted_scores = cropped_predictions[:, 1]  # Assuming class 1 corresponds to index 1 in predictions

# Normalize scores between 0 and 1
cropped_predicted_scores_normalized = (cropped_predicted_scores - np.min(cropped_predicted_scores)) / (np.max(cropped_predicted_scores) - np.min(cropped_predicted_scores))

In [ ]:
# Save predictions to a CSV file
output_file = 'predictions.csv'
with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['filename', 'liveness_score', 'liveness_score_crop'])  # Header
    for i, filename in enumerate(test_image_paths):
        writer.writerow([filename, predicted_scores_normalized[i], cropped_predicted_scores_normalized[i]])

print(f"Predictions saved to {output_file}")